# Sync with External Sensors

Many experimental setups record data from multiple sensors in parallel. This data needs to be synced temporally for a joint analysis. All eye tracking data you record with Pupil Invisible is accurately timestamped, which makes this easily possible.

In this guide you will learn how to sync eye tracking data from Pupil Invisible to any other timestamped external sensor using the `pd.merge_asof` function of Pandas. As an example, we will sync a heart rate sensor with a Pupil Invisible recording to produce a gaze overlay visualization with real-time heart rate of a man jogging.

TODO
- What heart rate sensor die we use?

## Dependencies of this Guide
To be written.
- Required Python libs
    - pandas av fitdecode tqdm opencv-python
- Where to download the code
- Where to download the example data
- How to put data into data folder

## Loading all Data
For the example visualization we need the scene video and gaze data from the Pupil Invisible recording, and the heart rate data from the [??]() recording.

The heart rate data can be read using the `fitdecode` module. For more details check out the implementation of the [load_fit_data]() implementation.

The gaze data CSV file can be read using Pandas. All we need from it is the timestamps and the gaze values.

For the scene video, we initially only need its timestamps and the corresponding frame indices for matching, we don't have to touch the actual video frames yet.

In [1]:
import pandas as pd

from decode_fit import load_fit_data


heart_rate_path = "data/eye-tracking-run.FIT"
hr = load_fit_data(heart_rate_path)
hr = hr[["timestamp", "heart_rate"]]

gaze_path = "data/demo-recording/running_rd-4a40d94d/gaze.csv"
gaze = pd.read_csv(gaze_path)
gaze["timestamp [ns]"] = pd.to_datetime(gaze["timestamp [ns]"])
gaze = gaze[["timestamp [ns]", "gaze x [px]", "gaze y [px]"]]

world_ts_path = "data/demo-recording/running_rd-4a40d94d/world_timestamps.csv"
world_ts = pd.read_csv(world_ts_path)
world_ts = world_ts[["timestamp [ns]"]]
world_ts["frame_index"] = world_ts.index
world_ts["timestamp [ns]"] = pd.to_datetime(world_ts["timestamp [ns]"])


## Timestamp Matching

The challenge with syncing the three data streams is that while they are all timestamped, their timestamps are not identical. Every stream is sampled independently and at different rates. E.g. the gaze data is sampled at 200 Hz, while the scene video is only sampled at 30 Hz, so there are a lot more gaze samples than video frames in our recording.

The visualization we are going for includes a gaze overlay, so given a scene video frame we need to overlay it with a gaze sample. We have to decide between two options here: 

**1.** Given the timestamp of the video frame we search for the gaze samples that is closest in time and choose it for the overlay. This would imply that most of the gaze samples are not visible in the overlay, because there are more gaze samples than frames.

**2.** We match every single gaze sample to its closest video frame. All the gaze samples that match to the same frame are averaged and this value is used for the overlay.

For gaze data option 2 is usually better, as the averaging contributes a bit of noise reduction.

The heart rate data is sampled much more sparsely with just 1 sample / minute. So for it we will choose option 1, which in this case means that the same heart rate sample will match to multiple video frames.

Both options can be implemented using the Pandas function [`pd.merge_asof`](https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.merge_asof.html). It merges two DataFrames based on indices that do not match perfectly by finding the closest matches, which is exactly what we need.

### Matching Video and Gaze
As mentioned above we will use option 2 for matching, i.e. we will match every gaze sample to the closest existing world frame, and then calculate the mean gaze value for every world frame. Note in the result below that the first couple of world frames do not have any matches. This is because the world camera initializes faster when starting a recording and thus records a couple seconds sooner than the eye cameras.

In [2]:
# Match every gaze sample to the closest world timestamp
gaze_world = pd.merge_asof(gaze, world_ts, left_on="timestamp [ns]", right_on="timestamp [ns]", direction="nearest")

# Calculate the mean gaze position for each world frame
gaze_world = gaze_world.groupby("frame_index").mean()

# Merge the gaze data with the world frame data
df = pd.merge(world_ts, gaze_world, left_on="frame_index", right_on="frame_index", how="left")
df

,timestamp [ns],frame_index,gaze x [px],gaze y [px]
0,2022-07-09 17:19:08.694000000,0,NaN,NaN
1,2022-07-09 17:19:08.744000000,1,NaN,NaN
2,2022-07-09 17:19:08.794000000,2,NaN,NaN
3,2022-07-09 17:19:08.844000000,3,NaN,NaN
4,2022-07-09 17:19:08.894000000,4,NaN,NaN
...,...,...,...,...
74214,2022-07-09 18:00:53.466044444,74214,427.126000,880.560833
74215,2022-07-09 18:00:53.498311111,74215,424.746857,882.799000
74216,2022-07-09 18:00:53.534600000,74216,426.106167,884.010000
74217,2022-07-09 18:00:53.566655555,74217,427.429429,884.170000


### Match Video and Heart Rate
We will match video and heart rate data using option 1, i.e. we match every world frame to the closest heart rate sample. Note how the heart rate samples repeat themselves for several world frames as expected.

In [3]:
# Match every world frame to the closest heart rate sample
world_hr = pd.merge_asof(world_ts, hr, left_on="timestamp [ns]", right_on="timestamp", direction="nearest")

# We use the frame index to merge this into the data frame
# so we can drop the timestamps here
world_hr.drop(["timestamp", "timestamp [ns]"], axis=1, inplace=True)

# Merge the matched heart rate data with the previous
# data frame containing the world frame data and gaze
df = pd.merge(df, world_hr, left_on="frame_index", right_on="frame_index", how="left")
df

,timestamp [ns],frame_index,gaze x [px],gaze y [px],heart_rate
0,2022-07-09 17:19:08.694000000,0,NaN,NaN,95
1,2022-07-09 17:19:08.744000000,1,NaN,NaN,95
2,2022-07-09 17:19:08.794000000,2,NaN,NaN,95
3,2022-07-09 17:19:08.844000000,3,NaN,NaN,95
4,2022-07-09 17:19:08.894000000,4,NaN,NaN,95
...,...,...,...,...,...
74214,2022-07-09 18:00:53.466044444,74214,427.126000,880.560833,169
74215,2022-07-09 18:00:53.498311111,74215,424.746857,882.799000,169
74216,2022-07-09 18:00:53.534600000,74216,426.106167,884.010000,169
74217,2022-07-09 18:00:53.566655555,74217,427.429429,884.170000,169


## Visualization
Now that we have matched all three data streams successfully we can visualize them together. This can be done in various ways and the complexity of creating visualizations is out of scope of this guide, but we have provided an example visualization below that shows a simple gaze overlay and textual visualization of the heart rate.

Feel free to check the implementation of the `make_visualization` function [here]().

In [4]:
from visualization import make_visualization

world_video_path = "data/demo-recording/running_rd-4a40d94d/3e2512bf_0.0-2504.94.mp4"
visualization_path = "data/visualization.mp4"


make_visualization(df, world_video_path, visualization_path)

nvenc not available h264_nvenc


 45%|████▌     | 33503/74219 [09:19<13:10, 51.51 frames/s]

TODO embed video of result